In [1]:
pip install -q langchain nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [4]:
# fixes a bug with asyncio and jupyter
import nest_asyncio
nest_asyncio.apply()


In [2]:
from langchain.document_loaders.sitemap import SitemapLoader


In [5]:
sitemap_loader = SitemapLoader(
    web_path="https://flywheeloutput.com/sitemap.xml")

docs = sitemap_loader.load()


In [7]:
len(docs)

96

In [8]:
docs[0]

Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nArchive - Flywheel DeFi\n\n\n\n\n\n\n\n\n\n\n\n\n\nFlywheel DeFiSubscribeSign inHomeArchiveAboutNewTopDiscussionThis Week in Frax - April 1st-7thGovernance passed its first major initiative for promoting Frax awareness, StakeDAO sdETH to remain locked, Bunni to receive gauges.Samuel McCulloch19 hr ago1Share this postThis Week in Frax - April 1st-7thflywheeloutput.comCopy linkTwitterFacebookEmailManifold Finance mevETH Interview with Head of BD JamesAn exclusive interview about the hottest new LSD coming to EthereumSamuel McCullochApr 611Share this postManifold Finance mevETH Interview with Head of BD Jamesflywheeloutput.comCopy linkTwitterFacebookEmail4 Things We Learned From Our Interview with DeFi SurferDeFi Surfer shares his thoughts on Arbitrum, Silvergate, DeFi Adoption, and his words of advice to FraxSamuel McCullochApr 5Share this post4 Things We Learned From Our Interview with DeFi Surferflywheeloutput.comCo

In [9]:
docs[10]

Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n"Finding Our Footing" - FraxCheck #32 - by kapital_k\n\n\n\n\n\n\n\n\n\n\n\n\n\nFlywheel DeFiSubscribeSign inShare this post"Finding Our Footing" - FraxCheck #32flywheeloutput.comCopy linkTwitterFacebookEmail"Finding Our Footing" - FraxCheck #32This week... I call this week the “Finding Our Footing” week. As the name suggests, this week we\'re trying to find solid ground. kapital_kMar 28Share this post"Finding Our Footing" - FraxCheck #32flywheeloutput.comCopy linkTwitterFacebookEmailTOP 3 TL;DRFraxBP - 114% of 3Pool, down 4% from last week’s FraxCheckFraxLend - Added two new pairs MKR and APE as well as a new v3 sfrxETH poolAMO - Currently at a loss of $180k, but could potentially be flat this month.Frax Supply & PegLet’s begin with the flagship product: FRAX stablecoin.  The supply remained flat this week.  However, the peg fluctuated violently between 1.002 and 0.991.  When checking the Curve markets, we swapped 

In [10]:
import re
from typing import List

HEADING_REGEX = re.compile(r'^#+ .*$')
WIDGET_REGEX = re.compile(r'^\$\$widget\d(.*?\))\$\$$')
MARKDOWN_LINK_REGEX = re.compile(r'^\[(.*)\]\(.*\)$')
MARKDOWN_IMAGE_REGEX = re.compile(r'^!\[.*\]\(.*\)$')
CITATION_REGEX = re.compile(
    r'\[\\\[\d+\\\]\]\(#cite-id-[a-z0-9]+\)', flags=re.MULTILINE)

MAX_PARA_COUNT = 3
MAX_CONTENT_LENGTH = 10000


def sanitize_content(content: str) -> str:
    """
    This function takes a string `content` that represents a Markdown document.
    It splits the document into paragraphs, filters out any paragraphs that do
    not contain valid text, removes Markdown syntax from the remaining paragraphs,
    and joins them back together into a string. The resulting string is trimmed
    to a maximum length and returned.
    """

    content_paragraphs = content.split('\n\n')
    filtered_paragraphs = [
        p for p in content_paragraphs if is_valid_paragraph(p)
    ]
    sanitized_paragraphs = []
    count = 0

    for paragraph in filtered_paragraphs:
        if count >= MAX_PARA_COUNT:
            break

        trimmed_paragraph = paragraph.strip()
        plain_text_paragraph = remove_markdown_syntax(trimmed_paragraph)
        sanitized_paragraph = CITATION_REGEX.sub('', plain_text_paragraph)
        is_heading = HEADING_REGEX.match(trimmed_paragraph) is not None

        if not is_heading:
            sanitized_paragraphs.append(sanitized_paragraph)
            count += 1
        else:
            sanitized_paragraphs.append(trimmed_paragraph)

    trimmed_content = '\n\n'.join(sanitized_paragraphs)[:MAX_CONTENT_LENGTH]

    return trimmed_content


def is_valid_paragraph(paragraph: str) -> bool:
    """
    This function takes a string `paragraph` and checks if it is a valid
    paragraph. A valid paragraph is one that contains text (i.e., is not empty),
    and does not contain Markdown image syntax, link syntax, or widget syntax.
    """

    para = paragraph.strip()
    return (
        len(para) != 0
        and not MARKDOWN_IMAGE_REGEX.match(para)
        and not MARKDOWN_LINK_REGEX.match(para)
        and not WIDGET_REGEX.match(para)
    )


def remove_markdown_syntax(text: str) -> str:
    """
    This function takes a string `paragraph` and checks if it is a valid
    paragraph. A valid paragraph is one that contains text (i.e., is not empty),
    and does not contain Markdown image syntax, link syntax, or widget syntax.
    """

    # Remove emphasis syntax
    text = re.sub(r'\*\*?(.*?)\*\*?', r'\1', text)
    text = re.sub(r'__(.*?)__', r'\1', text)

    # Remove code syntax
    text = re.sub(r'`(.+?)`', r'\1', text)

    # Remove links
    text = re.sub(r'\[(.*?)\]\((.*?)\)', r'\1', text)

    # Remove images
    text = re.sub(r'!\[(.*?)\]\((.*?)\)', r'', text)

    return text


def clean(string):
    sample = string

    cleaned = re.sub(r'\xa0', ' ',
                     re.sub(r'\s{2,}', ' ',
                            re.sub(r'\>', '',
                                   re.sub(r'\$\$widget0 YOUTUBE@VID\$\$', '',
                                          re.sub(r'[#*]+', '',
                                                 re.sub(r'\\\[\d+\\\]', '',
                                                        sample))))))
    return cleaned


In [21]:
type(docs[0])

langchain.schema.Document

In [23]:
sanitize_content(docs[0].page_content)

'Archive - Flywheel DeFi\n\nFlywheel DeFiSubscribeSign inHomeArchiveAboutNewTopDiscussionThis Week in Frax - April 1st-7thGovernance passed its first major initiative for promoting Frax awareness, StakeDAO sdETH to remain locked, Bunni to receive gauges.Samuel McCulloch19 hr ago1Share this postThis Week in Frax - April 1st-7thflywheeloutput.comCopy linkTwitterFacebookEmailManifold Finance mevETH Interview with Head of BD JamesAn exclusive interview about the hottest new LSD coming to EthereumSamuel McCullochApr 611Share this postManifold Finance mevETH Interview with Head of BD Jamesflywheeloutput.comCopy linkTwitterFacebookEmail4 Things We Learned From Our Interview with DeFi SurferDeFi Surfer shares his thoughts on Arbitrum, Silvergate, DeFi Adoption, and his words of advice to FraxSamuel McCullochApr 5Share this post4 Things We Learned From Our Interview with DeFi Surferflywheeloutput.comCopy linkTwitterFacebookEmail"Same Same But Different -- Again" - Frax Check #33This week... I c

In [24]:
clean(sanitize_content(docs[0].page_content))


'Archive - Flywheel DeFi Flywheel DeFiSubscribeSign inHomeArchiveAboutNewTopDiscussionThis Week in Frax - April 1st-7thGovernance passed its first major initiative for promoting Frax awareness, StakeDAO sdETH to remain locked, Bunni to receive gauges.Samuel McCulloch19 hr ago1Share this postThis Week in Frax - April 1st-7thflywheeloutput.comCopy linkTwitterFacebookEmailManifold Finance mevETH Interview with Head of BD JamesAn exclusive interview about the hottest new LSD coming to EthereumSamuel McCullochApr 611Share this postManifold Finance mevETH Interview with Head of BD Jamesflywheeloutput.comCopy linkTwitterFacebookEmail4 Things We Learned From Our Interview with DeFi SurferDeFi Surfer shares his thoughts on Arbitrum, Silvergate, DeFi Adoption, and his words of advice to FraxSamuel McCullochApr 5Share this post4 Things We Learned From Our Interview with DeFi Surferflywheeloutput.comCopy linkTwitterFacebookEmail"Same Same But Different -- Again" - Frax Check 33This week... I call 